2. Clasificación

En el dataset "drug200.csv" se encuentran los datos de medicamentos recetados para una determinada patología en función de las características de cada paciente.
- Separa en conjuntos de entrenamiento y test garantizando que haya muestras de todos los medicamentos en ambos conjuntos.
- Entrena un modelo que prediga el medicamento que se le recetará a un paciente en función de sus características.
- Crea un pipeline con los pasos seguidos.


In [13]:
# lib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline

# importar datos
df = pd.read_csv("./drug200.csv")


In [14]:
# data
print(type(df))
display(df)

<class 'pandas.core.frame.DataFrame'>


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [15]:
# detectamos los valores que no son numericos
print(df["Drug"].value_counts())
print("----------")
print(df["BP"].value_counts())
print("----------")
print(df["Cholesterol"].value_counts())
print("----------")
print(df["Sex"].value_counts())
print("----------")

Drug
DrugY    91
drugX    54
drugA    23
drugC    16
drugB    16
Name: count, dtype: int64
----------
BP
HIGH      77
LOW       64
NORMAL    59
Name: count, dtype: int64
----------
Cholesterol
HIGH      103
NORMAL     97
Name: count, dtype: int64
----------
Sex
M    104
F     96
Name: count, dtype: int64
----------


In [16]:
# creamos dos fuciones para comutar entre valores numericos y strings facilmente
# trasformamos los valores a num
def dfValToFloat(df):
    df_temp = df.copy()
    df_temp[["Drug"]] = df_temp[["Drug"]].replace("drugA",0.).replace("drugB",1.).replace("drugC",2.).replace("drugX",3.).replace("DrugY",4.)
    df_temp[["BP"]] = df_temp[["BP"]].replace("LOW",0.).replace("NORMAL",1.).replace("HIGH",2.)
    df_temp[["Cholesterol"]] = df_temp[["Cholesterol"]].replace("LOW",0.).replace("NORMAL",1.).replace("HIGH",2.)
    df_temp[["Sex"]] = df_temp[["Sex"]].replace("M",0.).replace("F",1.)
    return df_temp

# trasformamos los valores a string
def dfValToString(df):
    df_temp = df.copy()
    df_temp[["Drug"]] = df_temp[["Drug"]].replace(0.,"drugA").replace(1.,"drugB").replace(2.,"drugC").replace(3.,"drugX").replace(4.,"DrugY")
    df_temp[["BP"]] = df_temp[["BP"]].replace(0.,"LOW").replace(1.,"NORMAL").replace(2.,"HIGH")
    df_temp[["Cholesterol"]] = df_temp[["Cholesterol"]].replace(0.,"LOW").replace(1.,"NORMAL").replace(2.,"HIGH")
    df_temp[["Sex"]] = df_temp[["Sex"]].replace(0.,"M").replace(1.,"F")
    return df



In [17]:
dfNum = dfValToFloat(df)
display(dfNum)


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,1.0,2.0,2.0,25.355,4.0
1,47,0.0,0.0,2.0,13.093,2.0
2,47,0.0,0.0,2.0,10.114,2.0
3,28,1.0,1.0,2.0,7.798,3.0
4,61,1.0,0.0,2.0,18.043,4.0
...,...,...,...,...,...,...
195,56,1.0,0.0,2.0,11.567,2.0
196,16,0.0,0.0,2.0,12.006,2.0
197,52,0.0,1.0,2.0,9.894,3.0
198,23,0.0,1.0,1.0,14.020,3.0


In [18]:
print("no alteremos el df original")
display(df)

no alteremos el df original


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


In [19]:
# vemos que funciona la reconversion de datos
dfStr = dfValToString(df)
display(dfStr)
# verificamos que todos los datos son string
dfStr.dtypes

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


Age              int64
Sex             object
BP              object
Cholesterol     object
Na_to_K        float64
Drug            object
dtype: object

In [20]:
null_rows_idx = dfNum.isnull().any(axis=1)
print(null_rows_idx)
print(
    "Número de filas con valores nulos:", null_rows_idx.sum()
)  # Mostramos el número de filas con valores nulos

0      False
1      False
2      False
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199    False
Length: 200, dtype: bool
Número de filas con valores nulos: 0


In [21]:
# stratify - solucion errores 
# ya puede computar los valores 
# dado que son todos nuemericos
# verificamos que todos los datos son numericos
dfNum.dtypes


Age              int64
Sex            float64
BP             float64
Cholesterol    float64
Na_to_K        float64
Drug           float64
dtype: object

In [22]:
print("probamos stratify con valores solo numericos")
stratify = pd.cut (  # realiza un muestreo estratificado
            dfNum["Drug"],  # selecciono los datos de la columna "Drug"
            bins=[0., 1.5, 3.0, 4.5, 6., np.inf], # defino los límites 
            labels=[1, 2, 3, 4, 5] # defino etiquetas
          )
display(stratify)

probamos stratify con valores solo numericos


0      3
1      2
2      2
3      2
4      3
      ..
195    2
196    2
197    2
198    2
199    2
Name: Drug, Length: 200, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [23]:
# train_test_split con valores solo numeros [ValueError ]
# 
#     212 except InvalidParameterError as e:
#     213     # When the function is just a wrapper around an estimator, we allow
#     214     # the function to delegate validation to the estimator, but we replace
# ...
#     171         "#estimators-that-handle-nan-values"
#     172     )
# --> 173 raise ValueError(msg_err)
# 
# ValueError: Input y contains NaN.

# compruebo valores 
# no hay valores nan
print(dfNum.isna().sum())
# no hay valores nulos
print(dfNum.isnull().sum())

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64
Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64


In [24]:
print("probamos a crear datos de muestreo con [valores - numericos]")
# Separa en conjuntos de entrenamiento
# test garantizando que haya muestras de todos los medicamentos en ambos conjuntos.
train_set, test_set = train_test_split(
    dfNum,  # DataFrame
    test_size=0.2,  #  20% 
    random_state=42, # para que el resultado sea reproduccible
    stratify= pd.cut (  # realiza un muestreo estratificado
            dfNum["Drug"],  # selecciono los datos de la columna "Drug"
            bins=[0., 1.5, 3.0, 4.5, 6., np.inf], # defino los límites 
            labels=[1, 2, 3, 4, 5] # defino etiquetas
          )
)

probamos a crear datos de muestreo con [valores - numericos]


ValueError: Input y contains NaN.

In [ ]:
print("probamos a crear datos de muestreo con [valores - numericos]")
# Separa en conjuntos de entrenamiento
# test garantizando que haya muestras de todos los medicamentos en ambos conjuntos.
train_set, test_set = train_test_split(
    dfNum.drop(columns="BP"), # DataFrame
    dfNum["BP"], # tags
    # dfNum,  # DataFrame
    test_size=0.2,  #  20% 
    random_state=42, # para que el resultado sea reproduccible
    stratify= pd.cut (  # realiza un muestreo estratificado
            dfNum["Drug"],  # selecciono los datos de la columna "Drug"
            bins=[0., 1.5, 3.0, 4.5, 6., np.inf], # defino los límites 
            labels=[1, 2, 3, 4, 5] # defino etiquetas
          )
)

probamos a crear datos de muestreo con [valores - numericos]


ValueError: Input y contains NaN.

In [ ]:
print("probamos a crear datos de muestreo con [valores - sin modificar]")
# Separa en conjuntos de entrenamiento
# test garantizando que haya muestras de todos los medicamentos en ambos conjuntos.
train_set, test_set = train_test_split(
    df.drop(columns="BP"), # DataFrame
    df["BP"], # tags
    # dfNum,  # DataFrame
    test_size=0.2,  #  20% 
    random_state=42, # para que el resultado sea reproduccible
    stratify= pd.cut (  # realiza un muestreo estratificado
            df["BP"],  # selecciono los datos de la columna "Drug"
            bins=[0., 1.5, 3.0, 4.5, 6., np.inf], # defino los límites 
            labels=[1, 2, 3, 4, 5] # defino etiquetas
          )
)

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64
Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64


In [ ]:


# Separa en conjuntos de entrenamiento
# test garantizando que haya muestras de todos los medicamentos en ambos conjuntos.
train_set, test_set = train_test_split(
    df.drop(columns="BP"), # DataFrame
    df["BP"], # tags
    # dfNum,  # DataFrame
    test_size=0.2,  #  20% 
    random_state=42, # para que el resultado sea reproduccible
    stratify= pd.cut (  # realiza un muestreo estratificado
            df["BP"],  # selecciono los datos de la columna "Drug"
            bins=[0., 1.5, 3.0, 4.5, 6., np.inf], # defino los límites 
            labels=[1, 2, 3, 4, 5] # defino etiquetas
          )
)

ValueError: too many values to unpack (expected 2)